In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import itertools
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
from tqdm import tqdm
import re
from nltk.corpus import stopwords
from razdel import tokenize 
import pymorphy2


%matplotlib inline

In [2]:
df = pd.read_excel('item.xlsx')

In [3]:
df

,Период,Номенклатура,Контрагент,Цена
0,14.01.2021 14:22:17,Нож технический 18мм,ЦентрСнаб,20.10
1,14.01.2021 14:22:17,"Пылесос строительный MAKITA VC2512L (1000 Вт, ...",ЦентрСнаб,13378.00
2,14.01.2021 14:22:17,Флисовый пылесборный мешок для VC2010L/VC3011L...,ЦентрСнаб,1757.70
3,14.01.2021 14:22:17,"Штроборез MAKITA SG150 (1800 Вт, 150мм, 7-45мм...",ЦентрСнаб,52980.00
4,14.01.2021 14:19:25,Лента ACE-TAPE/50 Armaflex,ГЕСЕР,894.00
...,...,...,...,...
208356,25.01.2013 17:24:07,Карандаш строительный 180мм,Боргус,6.00
208357,25.01.2013 17:24:07,Силикон прозрачный,Боргус,95.00
208358,25.01.2013 17:24:07,"Круг отрезной по металлу 125х1,2х22,2мм",Боргус,9.95
208359,25.01.2013 17:24:07,Бур SDS-plus 8х100х160 мм,Боргус,54.00


In [4]:
df_item=pd.DataFrame(df['Номенклатура'])

In [5]:
df_item.drop_duplicates('Номенклатура',inplace=True, ignore_index=True)

In [6]:
stopword_ru = stopwords.words('russian')
morph = pymorphy2.MorphAnalyzer()

In [7]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))
    text = re.sub("[[]|[]]|[/]", ' ', text)
    text = re.sub("[-—.,_:;*()]", ' ', text)
    text = re.sub(r"(\d)(x)(\d)", r'\1 \3', text)
    text = re.sub(r"(\d)(X)(\d)", r'\1 \3', text)
    text = re.sub(r"(\d)(х)(\d)", r'\1 \3', text)
    text = re.sub(r"(\d)(Х)(\d)", r'\1 \3', text)
    text = re.sub(" х | Х | x | X ", ' ', text)
    
    
    text = re.sub("[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'["]','',text)
    text = re.sub(r"['”]",'',text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    text = re.sub("(\d)(\D)",r'\1 \2',text)
    text = re.sub("(\D)(\d)",r'\1 \2',text)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if w in cache: # [4]
            words_lem.append(cache[w])
        else: # [5]
            temp_cach = cache[w] = morph.parse(w)[0].normal_form
            words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return ' '.join(words_lem_without_stopwords)


def get_search_word(txt):
    search_word = clean_text(txt)
    search_word = lemmatization(search_word)
    return  text.transform ([search_word])

def search_item(search_word,num_var):
    search_word = get_search_word(search_word)
    _,w = search_word.nonzero()
    w_w = search_word.data
    search_data = []
    for i in range(dict_text.shape[0]):
        d = 0
        inx_row  = np.where(row==i)
        string = word[inx_row]
        string_data = words_weights[inx_row]
        for k,j in enumerate(w):
            if j in string:
                 d+= (w_w[k]-string_data[np.where(string==j)][0])**2
            else:
                 d+= w_w[k]**2  
        search_data.append((i,d**0.5))        
    search_data.sort(key = lambda x: x[1])
    my_words= list(map(lambda x: df_item.loc[x[0],'Номенклатура'] ,search_data[:num_var]))
    return my_words


In [8]:
df_item['analys']=df_item['Номенклатура']

In [9]:
%%time
df_item['analys']=df_item['Номенклатура'].apply(lambda x:clean_text(x) )

<ipython-input-7-3ad013ee418f>:14: FutureWarning: Possible nested set at position 1
  text = re.sub("[[]|[]]|[/]", ' ', text)
<ipython-input-7-3ad013ee418f>:23: FutureWarning: Possible nested set at position 33
  text = re.sub("[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)


Wall time: 1.31 s


In [10]:
%%time
df_item['analys']=df_item['analys'].apply(lambda x:lemmatization(x) )

Wall time: 3.48 s


In [11]:
df_item

,Номенклатура,analys
0,Нож технический 18мм,нож технический 18 мм
1,"Пылесос строительный MAKITA VC2512L (1000 Вт, ...",пылесос строительный makita vc 2512 l 1000 вт ...
2,Флисовый пылесборный мешок для VC2010L/VC3011L...,флисовыя пылесборный мешок vc 2010 l vc 3011 l...
3,"Штроборез MAKITA SG150 (1800 Вт, 150мм, 7-45мм...",штроборез makita sg 150 1800 вт 150 мм 7 45 мм...
4,Лента ACE-TAPE/50 Armaflex,лента ace tape 50 armaflex
...,...,...
31405,Напильник 3-гр 300 мм № 2,напильник 3 гр 300 мм 2
31406,Ключ накидной односторонний 80мм ударный,ключ накидной односторонний 80 мм ударный
31407,Замок-защелка регулируемый Тип 3 арт. 112,замок защёлка регулировать тип 3 арт 112
31408,Лента самоклеющаяся уплотнительная 5х10,лента самоклеющийся уплотнительный 5 10


In [12]:
text = TfidfVectorizer()

In [13]:
dict_text = text.fit_transform(df_item['analys'])

In [14]:
dict_text.shape

(31410, 13697)

In [15]:
words_weights = dict_text.data

In [16]:
row,word = dict_text.nonzero()

In [17]:
%%time
search_item('Диск алмазный Stnd Concrete 125-22.23',5)

Wall time: 5 s


['Алмазный диск  BOSCH Pf Concrete 125 х 22 мм по б',
 'Алмазный диск BOSCH Stf Concrete 125 х 22 мм',
 'Алмазный диск BOSCH Pf Concrete 125 х 22 мм по бетону',
 'Алмазный диск сегментный BOSCH Pf Concrete 125 х 22 мм по бетону',
 'Алмазный диск BOSCH Stf Concrete 230 х 22 мм']

In [18]:
%%time
search_item('Перчатки ХБ с ПВХ 4-х ниточные',3)

Wall time: 4.75 s


['Перчатки ХБ с ПВХ 5-х ниточные',
 'Перчатки ХБ с ПВХ 3-х ниточные',
 'Перчатки ХБ с ПВХ 4-х ниточные']

In [19]:
%%time
search_item('Профиль монтажный U-образный 40*40*2,5',10)

Wall time: 4.79 s


['Профиль монтажный L-образный 30х30х2000х2',
 'Профиль монтажный П-образный (30*30*2000*2)',
 'Скоба U образная 1/2" (D25*H40) с двумя гайками M6',
 'Профиль монтажный U-образный 30х30х30х2000х2',
 'Профиль монтажный П-образный 30*30*30*2мм L=2000мм',
 'Профиль монтажный U-образный 30*30*30*2мм ПП-U L=3000мм',
 'Профиль монтажный П-образный 30*30*30*2мм L=2000мм отверстия с 3-х сторон',
 'Коробка монтажная 60х40мм',
 'Профиль направляющий ПН-2 50х40 (3м)',
 'Профиль направляющий ПН-2 50х40 3м']

In [21]:
%%time
search_item('Анкер латунный диам 10',10)

Wall time: 4.76 s


['Анкер латунный цанга М10',
 'Анкер латунный забивной М10',
 'Анкер латунный забивной М10 (уп. 3 шт)',
 'Анкер латунный М10 РМА',
 'Анкер латунный MSA 10 Sormat',
 'Анкер латунный забивной М14',
 'Анкер латунный забивной М16',
 'Анкер латунный забивной М12',
 'Анкер латунный забивной М8',
 'Анкер латунный забивной М6']